In [ ]:
import pandas as pd
import numpy as np

In [29]:
data = pd.read_csv("dirty_cafe_sales.csv")

In [30]:
#information about the cafe sales dataset
print("Cafe Sales Dataset:")
print("\n\nHead about the dataset:")
print(data.head())
print("----------------------------------------")
print("\n\nShape about the dataset:")
print(data.shape)
print("----------------------------------------")
print("\n\nColumns about the dataset:")
print(data.columns)
print("\n\nChecking for missing values in each column:")
print(data.isnull().sum())
print("----------------------------------------")
print("\n\nNumber of duplicated rows in the dataset:")
print(data.duplicated().sum())
print("----------------------------------------")
print("\n\nInformation about the dataset:")
print(data.info())
print("----------------------------------------")
print("\n\nStatistical summary of the dataset:")
print(data.describe())
print("----------------------------------------")
print("\n\nUnique values count in each column:")
for column in data.columns:
    print(f"{column}: {data[column].nunique()}")
print("----------------------------------------")
print("\n\nUnique values in each column:")
for column in data.columns:
    print(f"{column}: {data[column].unique()}")

Cafe Sales Dataset:


Head about the dataset:
  Transaction ID    Item Quantity Price Per Unit Total Spent  Payment Method  \
0    TXN_1961373  Coffee        2            2.0         4.0     Credit Card   
1    TXN_4977031    Cake        4            3.0        12.0            Cash   
2    TXN_4271903  Cookie        4            1.0       ERROR     Credit Card   
3    TXN_7034554   Salad        2            5.0        10.0         UNKNOWN   
4    TXN_3160411  Coffee        2            2.0         4.0  Digital Wallet   

   Location Transaction Date  
0  Takeaway       2023-09-08  
1  In-store       2023-05-16  
2  In-store       2023-07-19  
3   UNKNOWN       2023-04-27  
4  In-store       2023-06-11  
----------------------------------------


Shape about the dataset:
(10000, 8)
----------------------------------------


Columns about the dataset:
Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
  

In [31]:
#Replacing 'Unknown' & 'Errors' by NaN
data = data.replace(['ERROR', 'UNKNOWN'], np.nan)

In [32]:
#Converting columns into numeric data type
numeric_cols = ['Quantity', 'Price Per Unit', 'Total Spent']
for col in numeric_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')


In [33]:
#Converting column date into DateTime Format
data['Transaction Date'] = pd.to_datetime(
    data['Transaction Date'],
    errors='coerce'
)


In [34]:
#Cleaning categorical columns
categorical_cols = ['Item', 'Payment Method', 'Location']
for col in categorical_cols:
    data[col] = data[col].str.strip().str.title()


In [35]:
#Removing invalid transactions
data = data.dropna(subset=['Item', 'Quantity', 'Price Per Unit'])
data = data[
    (data['Quantity'] > 0) &
    (data['Price Per Unit'] > 0)
]


In [36]:
#Fixing & Recalculating Total Spent Column
data['Calculated Total'] = data['Quantity'] * data['Price Per Unit']
data['Total Spent'] = data['Total Spent'].fillna(data['Calculated Total'])
data = data.drop(columns='Calculated Total')


In [ ]:
#Handling Missing Payment Method & Location by filling 'Unknown'
data['Payment Method'] = data['Payment Method'].fillna('Unknown')
data['Location'] = data['Location'].fillna('Unknown')

In [38]:
#Data Validation
assert data['Quantity'].min() > 0
assert data['Price Per Unit'].min() > 0
assert data['Total Spent'].min() > 0
assert data['Transaction Date'].notna().sum() > 0

In [41]:
#Verifying unique data values in each columns
for i in data.columns:
    print(f"{i}: {data[i].unique()}")
    print("------------------------------------------------")

Transaction ID: ['TXN_1961373' 'TXN_4977031' 'TXN_4271903' ... 'TXN_7672686' 'TXN_5255387'
 'TXN_6170729']
------------------------------------------------
Item: ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'Sandwich' 'Juice' 'Tea']
------------------------------------------------
Quantity: [2. 4. 5. 3. 1.]
------------------------------------------------
Price Per Unit: [2.  3.  1.  5.  4.  1.5]
------------------------------------------------
Total Spent: [ 4.  12.  10.  20.  16.  25.   8.   5.  15.   6.   3.   2.   9.   1.
  7.5  1.5  4.5]
------------------------------------------------
Payment Method: ['Credit Card' 'Cash' 'Unknown' 'Digital Wallet']
------------------------------------------------
Location: ['Takeaway' 'In-Store' 'Unknown']
------------------------------------------------
Transaction Date: <DatetimeArray>
['2023-09-08 00:00:00', '2023-05-16 00:00:00', '2023-07-19 00:00:00',
 '2023-04-27 00:00:00', '2023-06-11 00:00:00', '2023-03-31 00:00:00',
 '2023-10-28 00:00:0

In [42]:
#Drop rows having missing date
data = data.dropna(subset=['Transaction Date'])

In [44]:
#Enforce Revenue Consistency
revenue_mismatch = (
    data['Total Spent'] != data['Quantity'] * data['Price Per Unit']
)
revenue_mismatch.sum()

np.int64(0)

In [45]:
#Optimize Data Types
data['Item'] = data['Item'].astype('category')
data['Payment Method'] = data['Payment Method'].astype('category')
data['Location'] = data['Location'].astype('category')

In [46]:
#Exporting CSV files
data.to_csv("cafe_sales.csv", index=False)
